## Data Loader

In [14]:
from torch.utils.data import Dataset ,DataLoader

class GPTDatav1(Dataset):
  def __init__(self,txt,tokenizer ,max_length ,stride):
    self.input_ids = []
    self.target_ids = []


    token_ids = tokenizer.encode(txt,allowed_special={"<|endoftext|>"})

    for i in range(0,len(token_ids)-max_length,stride):
      input_chunk = token_ids[i:i+max_length]
      target_chunk = token_ids[i+1:i+max_length+1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self,idx):
    return self.input_ids[idx] , self.target_ids[idx]

In [15]:
!pip install tiktoken

In [16]:
import tiktoken
def create_dataloader(txt,batch_size =4 ,
                      max_length = 256 ,stride = 128 ,
                      shuffle =True,drop_last =True ,
                      num_workers = 0):
  tokenizer = tiktoken.get_encoding("gpt2")
  dataset = GPTDatav1(txt,tokenizer,max_length,stride)
  dataloader = DataLoader(dataset,batch_size=batch_size,
                          shuffle=shuffle,
                          drop_last=drop_last,
                          num_workers=num_workers)
  return dataloader

In [17]:
with open("/content/verdict.txt","r",encoding="utf-8") as f:
  raw_data = f.read()




In [19]:
import torch

dataloader = create_dataloader(
    raw_data,batch_size=1,max_length=4,stride=1,shuffle = False
)


data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)



[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [20]:
second_batch = iter(data_iter)
print(second_batch)

In [23]:
dataloader = create_dataloader(
    raw_data,batch_size=8,max_length=4,stride=4,shuffle = False
)

data_iter = iter(dataloader)
inputs,targets =next(data_iter)
print(inputs)
print(targets)

tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## Token Embeddings

In [27]:
input_ids = torch.tensor([2,3,5,1])


vocab_size =6
output_dim = 3

torch.manual_seed(123)

embedding_layer = torch.nn.Embedding(vocab_size,output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [28]:
print(embedding_layer(input_ids))


tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


## Positional Encoding

In [37]:
vocab_size = 50257
output_dim= 256

token_embedding_layer =torch.nn.Embedding(vocab_size,output_dim)

max_length= 4
data_loader = create_dataloader(
    raw_data,batch_size=8,max_length=max_length,stride=1,shuffle = False
)

data_iter =iter(data_loader)
inputs,targets = next(data_iter)

print(inputs.shape)
print(inputs)


torch.Size([8, 4])
tensor([[   40,   367,  2885,  1464],
        [  367,  2885,  1464,  1807],
        [ 2885,  1464,  1807,  3619],
        [ 1464,  1807,  3619,   402],
        [ 1807,  3619,   402,   271],
        [ 3619,   402,   271, 10899],
        [  402,   271, 10899,  2138],
        [  271, 10899,  2138,   257]])


In [41]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)


torch.Size([8, 4, 256])


In [42]:
context_length = max_length

pos_embedding_layer = torch.nn.Embedding(context_length,output_dim)

pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [43]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
